<a href="https://colab.research.google.com/github/AntarikshVerma/Prompt_Engineering/blob/main/prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prompting Principles
1. Principle 1: Write clear and specific instructions
2. Principle 2: Give the model time to “think”


##Tactics

##Azure OPEN AI Setup

In [4]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [6]:
from openai import AzureOpenAI
import os

from google.colab import userdata
open_ai_key=userdata.get('OPENAI_API_KEY')
open_ai_base=userdata.get('OPEN_API_URL')
api_version="2023-07-01-preview"

In [7]:
openai = AzureOpenAI(
            api_version=api_version,
            azure_endpoint=open_ai_base,
            api_key=open_ai_key,
        )

In [8]:
def get_completion(prompt, model="gpt-35-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content


##Tactic 1: Use delimiters to clearly indicate distinct parts of the input
###Delimiters can be anything like: ```, """, < >, <tag> </tag>, :

In [9]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

It is important to provide clear and specific instructions to guide the model towards the desired output and reduce the chances of receiving irrelevant or incorrect responses, and longer prompts can provide more clarity and context for the model.


##Tactic 2: Ask for a structured output
####JSON, HTML

In [10]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

{
  "books": [
    {
      "book_id": 1,
      "title": "The Midnight Garden",
      "author": "Eleanor Black",
      "genre": "Fantasy"
    },
    {
      "book_id": 2,
      "title": "The Forgotten Symphony",
      "author": "Lucas Hart",
      "genre": "Mystery"
    },
    {
      "book_id": 3,
      "title": "Echoes of Eternity",
      "author": "Aria Nightingale",
      "genre": "Romance"
    }
  ]
}
